In [1]:
import pandas as pd

basics = pd.read_csv('title.basics.tsv', sep='\t', low_memory=False, na_values=["\\N","nan"])
basics = basics[basics["titleType"] == "movie"]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0.0,1894.0,NaN,45,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0.0,1897.0,NaN,100,"Documentary,News,Sport"
498,tt0000502,movie,Bohemios,Bohemios,0.0,1905.0,NaN,100,NaN
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0.0,1906.0,NaN,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0.0,1907.0,NaN,90,Drama
...,...,...,...,...,...,...,...,...,...
9629481,tt9916622,movie,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,0.0,2015.0,NaN,57,Documentary
9629508,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,De la ilusión al desconcierto: cine colombiano...,0.0,2007.0,NaN,100,Documentary
9629520,tt9916706,movie,Dankyavar Danka,Dankyavar Danka,0.0,2013.0,NaN,NaN,Comedy
9629530,tt9916730,movie,6 Gunn,6 Gunn,0.0,2017.0,NaN,116,NaN


In [2]:
ratings = pd.read_csv('title.ratings.tsv', sep='\t', low_memory=False, na_values=["\\N","nan"])
ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,1952
1,tt0000002,5.8,264
2,tt0000003,6.5,1787
3,tt0000004,5.6,179
4,tt0000005,6.2,2589
...,...,...,...
1281760,tt9916730,8.3,10
1281761,tt9916766,7.0,21
1281762,tt9916778,7.2,36
1281763,tt9916840,8.8,6


In [3]:
cast = pd.read_csv('title.principals.tsv', sep='\t', low_memory=False, na_values=["\\N","nan"])
names = pd.read_csv('name.basics.tsv', sep='\t', low_memory=False, na_values=["\\N","nan"])

In [4]:
df = pd.merge(basics, ratings, on='tconst')
df = df.sort_values(by='numVotes', ascending=False)

top = df.loc[df['primaryTitle'] == df['originalTitle']].head(2000).merge(
    cast.loc[cast['category'].isin(['actor','actress'])], on='tconst'
).merge(names, on='nconst').groupby(['tconst']).agg({
    'primaryName': '|'.join
}).merge(basics, on='tconst')

top

,tconst,primaryName,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0012349,Charles Chaplin|Henry Bergman|B.F. Blinn|Edna ...,movie,The Kid,The Kid,0.0,1921.0,NaN,68,"Comedy,Drama,Family"
1,tt0015864,Charles Chaplin|Henry Bergman|Frank Aderias|Ma...,movie,The Gold Rush,The Gold Rush,0.0,1925.0,NaN,95,"Adventure,Comedy,Drama"
2,tt0017136,Brigitte Helm|Alfred Abel|Gustav Fröhlich|Rudo...,movie,Metropolis,Metropolis,0.0,1927.0,NaN,153,"Drama,Sci-Fi"
3,tt0021749,Charles Chaplin|Virginia Cherrill|Florence Lee...,movie,City Lights,City Lights,0.0,1931.0,NaN,87,"Comedy,Drama,Romance"
4,tt0027977,Al Ernest Garcia|Charles Chaplin|Paulette Godd...,movie,Modern Times,Modern Times,0.0,1936.0,NaN,87,"Comedy,Drama,Romance"
...,...,...,...,...,...,...,...,...,...,...
1992,tt9639470,Anya Taylor-Joy|Thomasin McKenzie|Matt Smith|D...,movie,Last Night in Soho,Last Night in Soho,0.0,2021.0,NaN,116,"Drama,Horror,Mystery"
1993,tt9764362,Ralph Fiennes|Nicholas Hoult|Anya Taylor-Joy|H...,movie,The Menu,The Menu,0.0,2022.0,NaN,107,"Horror,Thriller"
1994,tt9770150,Frances McDormand|David Strathairn|Linda May|G...,movie,Nomadland,Nomadland,0.0,2020.0,NaN,107,Drama
1995,tt9777666,Chris Pratt|J.K. Simmons|Yvonne Strahovski|Bet...,movie,The Tomorrow War,The Tomorrow War,0.0,2021.0,NaN,138,"Action,Adventure,Drama"


In [5]:
from rdflib import Graph
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://localhost:7200/repositories/wikidata")

def gen():
    QUERY = '''
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    SELECT DISTINCT
    ?title ?tconst
    (GROUP_CONCAT(DISTINCT ?director; SEPARATOR="|") as ?directors)
    (GROUP_CONCAT(DISTINCT ?composer; SEPARATOR="|") as ?composers)
    (GROUP_CONCAT(DISTINCT ?genre; SEPARATOR="|") as ?genres)
    (GROUP_CONCAT(DISTINCT ?subject; SEPARATOR="|") as ?subjects)
    (GROUP_CONCAT(DISTINCT ?period; SEPARATOR="|") as ?periods)
    (GROUP_CONCAT(DISTINCT ?location; SEPARATOR="|") as ?locations)

    WHERE {
        ?film wdt:P31 wd:Q11424 .
        ?film rdfs:label ?title .
        ?film wdt:P345 ?tconst .
        OPTIONAL { ?film wdt:P57 ?d . ?d rdfs:label ?director . }
        OPTIONAL { ?film wdt:P86 ?c . ?c rdfs:label ?composer . }
        OPTIONAL { ?film wdt:P136 ?g . ?g rdfs:label ?genre . }
        OPTIONAL { ?film wdt:P921 ?s . ?s rdfs:label ?subject . }
        OPTIONAL { ?film wdt:P180 ?s2 . ?s2 rdfs:label ?subject . }
        OPTIONAL { ?film wdt:P2401 ?p . ?p rdfs:label ?period . }
        OPTIONAL { ?film wdt:P840 ?l . ?l rdfs:label ?location . }
    }
    GROUP BY ?title ?tconst
    '''

    sparql.setQuery(QUERY)
    sparql.setReturnFormat(JSON)
    ret = sparql.queryAndConvert()

    for row in ret['results']['bindings']:
        yield {k: v['value'] for k, v in row.items()}


df2 = pd.DataFrame.from_records(gen())
df2

,title,tconst,directors,composers,genres,subjects,periods,locations
0,The Intouchables,tt1675434,Éric Toledano|Olivier Nakache,Ludovico Einaudi,film based on literature|biographical film|com...,,,Paris
1,Copernicus,tt0070278,Czesław Petelski|Ewa Petelska,Jerzy Maksymiuk,historical film,,,
2,Ivan's Childhood,tt0056111,Andrei Tarkovsky,Vyacheslav Ovchinnikov,drama film|war film,World War II,,Soviet Union
3,Andrei Rublev,tt0060107,Andrei Tarkovsky,Vyacheslav Ovchinnikov,drama film|art film|medieval film,,,Moscow
4,The Sacrifice,tt0091670,Andrei Tarkovsky,Johann Sebastian Bach,drama film,World War III|religiosity|sacrifice,,Sweden
...,...,...,...,...,...,...,...,...
105129,The Truth About Husbands,tt0011794,Kenneth Webb,,drama film|silent film,,,
105130,The Postman,tt0299582,Kamel El-Telmissany,,,,,
105131,Princess Leila,tt0273730,Niazi Mostafa,,,,,
105132,Served Like a Girl,tt6175710,,Michael A. Levine,documentary film,,,


In [6]:
df3 = top.merge(df2, on='tconst')
df3

,tconst,primaryName,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres_x,title,directors,composers,genres_y,subjects,periods,locations
0,tt0015864,Charles Chaplin|Henry Bergman|Frank Aderias|Ma...,movie,The Gold Rush,The Gold Rush,0.0,1925.0,NaN,95,"Adventure,Comedy,Drama",The Gold Rush,Charlie Chaplin,Charlie Chaplin,comedy film|silent film|romantic comedy|social...,,,
1,tt0017136,Brigitte Helm|Alfred Abel|Gustav Fröhlich|Rudo...,movie,Metropolis,Metropolis,0.0,1927.0,NaN,153,"Drama,Sci-Fi",Metropolis,Fritz Lang,Gottfried Huppertz,drama film|fantasy film|silent film|film based...,love|technology|working class|dehumanization|r...,,
2,tt0021749,Charles Chaplin|Virginia Cherrill|Florence Lee...,movie,City Lights,City Lights,0.0,1931.0,NaN,87,"Comedy,Drama,Romance",City Lights,Charlie Chaplin,Charlie Chaplin,tragicomedy|comedy drama|romantic comedy,,,
3,tt0031381,Thomas Mitchell|Clark Gable|Vivien Leigh|Barba...,movie,Gone with the Wind,Gone with the Wind,0.0,1939.0,NaN,238,"Drama,Romance,War",Gone with the Wind,Sam Wood|George Cukor|Victor Fleming,Max Steiner,drama film|romance film|film based on a novel|...,slavery in the United States|revenge,,London|Atlanta
4,tt0032138,Judy Garland|Frank Morgan|Ray Bolger|Bert Lahr,movie,The Wizard of Oz,The Wizard of Oz,0.0,1939.0,NaN,102,"Adventure,Family,Fantasy",The Wizard of Oz,King Vidor|Victor Fleming,Harold Arlen,drama film|fantasy film|film based on a novel|...,,,Kansas|Land of Oz|Emerald City|Munchkin Country
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122,tt9419884,Benedict Cumberbatch|Chiwetel Ejiofor|Benedict...,movie,Doctor Strange in the Multiverse of Madness,Doctor Strange in the Multiverse of Madness,0.0,2022.0,NaN,126,"Action,Adventure,Fantasy",Doctor Strange in the Multiverse of Madness,Sam Raimi,Danny Elfman,horror film|action film|fantasy film|adventure...,,,
1123,tt9484998,J.K. Simmons|Andy Samberg|Cristin Milioti|Pete...,movie,Palm Springs,Palm Springs,0.0,2020.0,NaN,90,"Comedy,Fantasy,Mystery",Palm Springs,Max Barbakow,,comedy film,time loop,,Palm Springs
1124,tt9495224,Fionn Whitehead|Craig Parkinson|Alice Lowe|Asi...,movie,Black Mirror: Bandersnatch,Black Mirror: Bandersnatch,0.0,2018.0,NaN,90,"Drama,Fantasy,Mystery",Black Mirror: Bandersnatch,David Slade,Brian Reitzell,drama film|science fiction film,,,London
1125,tt9770150,Frances McDormand|David Strathairn|Linda May|G...,movie,Nomadland,Nomadland,0.0,2020.0,NaN,107,Drama,Nomadland,Chloé Zhao,Ludovico Einaudi,drama film|road movie,autonomy|self-actualization|lifestyle|precaria...,,Arizona|Colorado|California|Nebraska|Nevada|Ba...


In [7]:
import json 

film_data = {}

for row in df3.itertuples():
    title = row.title
    year = int(row.startYear)
    display = f'{title} ({year})'
    film = {
        'title': title,
        'imdb': row.tconst,
        'decade': str(int(year / 10) * 10) + 's',
        'actors': sorted(set(
            row.primaryName.split('|')
        )),
    }

    film['directors'] = sorted(set(row.directors.split('|')), key=str.casefold) or None
    film['composers'] = sorted(filter(bool, set(row.composers.split('|'))), key=str.casefold) or None
    film['settings'] = sorted(filter(bool, set(row.periods.split('|')) | set(row.locations.split('|'))),
        key=str.casefold
    ) or None
    film['genres'] = sorted(
        set(g.replace(' film', '').lower() for g in row.genres_y.split('|'))
        |
        set(g.lower() for g in row.genres_x.split(',')),
        key=str.casefold
    ) or None
    film['subjects'] = sorted(filter(bool, set(row.subjects.split('|'))), key=str.casefold) or None

    if len(film) > 2:
        film_data[display] = film

with open('films.json', 'w') as f:
    json.dump(film_data, f, indent=2)